1) Installing extensions and importing libraries necessary for reading HTML and wrangling data

In [68]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [69]:
import pandas as pd 
import numpy as np
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Matplotlib and associated plotting modules
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
import json
print('imported')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

imported


2) Loading data from Wikipedia link by using pandas read_html and saving to dataframe toronto

In [70]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
rawFrame = pd.read_html(link) 
toronto = rawFrame[0]

3) Dropping all rows with unassigned Borough and then replacing unassigned Neighborhoods with assigned Boroughs

In [71]:
toronto.drop(toronto.loc[toronto['Borough']=='Not assigned'].
             index, inplace=True)
toronto.Neighbourhood.replace('Not assigned', toronto.Borough, inplace=True)
toronto.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


4) Renaming columns to match the example dataframe

In [72]:
toronto.columns = ['PostalCode', 'Borough', 'Neighborhood']

5) Grouping by PostalCode and allowing multiple values in Neighborhood column

In [73]:
df_toronto = toronto.groupby(['PostalCode','Borough'])['Neighborhood'].agg(lambda col:', '.join(col)).reset_index()
df_toronto.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [74]:
df_toronto.shape

(103, 3)

6)Loading coordinate data from CSV file to pandas dataframe

In [75]:
lat_long = pd.read_csv('/resources/Geospatial_Coordinates.csv')
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


7)Combining coordinate dataframe with df_toronto and dropping duplicate columns

In [76]:
latLong_toronto = pd.concat([df_toronto, lat_long], axis=1).T.drop_duplicates().T
latLong_toronto.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648


8)Creating a map of Toronto with folium using data from latLong_toronto

In [77]:
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)
for lat, lng, borough, neighborhood in zip(latLong_toronto['Latitude'], latLong_toronto['Longitude'], latLong_toronto['Borough'], latLong_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='red',
        fill=True,
        fill_color='#769d96',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

9) Mapping a single Borough for Foursquare examination

In [78]:
scar_data = latLong_toronto[latLong_toronto['Borough']=='Scarborough'].reset_index(drop=True)
scar_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


In [79]:
la = 43.7764
lo = -79.2318
map_scarborough = folium.Map(location=[la, lo], zoom_start=12)
map_scarborough

10) Loading Foursquare credentials

In [80]:
CLIENT_ID = 'RBTEPRZM3KZMHSWZCOZELLIEC1YN2ZFSWCVH4FTOXPZZXRZH' # your Foursquare ID
CLIENT_SECRET = 'RQMYMSAHQKFM1F13BWWW4CTKIKIXJRTCMO5JCJJTCK4FBDYL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentail: ' + CLIENT_ID)
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentail: RBTEPRZM3KZMHSWZCOZELLIEC1YN2ZFSWCVH4FTOXPZZXRZH
CLIENT_ID: RBTEPRZM3KZMHSWZCOZELLIEC1YN2ZFSWCVH4FTOXPZZXRZH
CLIENT_SECRET:RQMYMSAHQKFM1F13BWWW4CTKIKIXJRTCMO5JCJJTCK4FBDYL


In [81]:
scar_data.loc[4,'Neighborhood']

'Cedarbrae'

In [82]:
hood_lat = scar_data.loc[4, 'Latitude']
hood_long = scar_data.loc[4, 'Longitude']
hood_name = scar_data.loc[4, 'Neighborhood']
print('Coordinates for {} are {}, {}.'.format(hood_name,
                                             hood_lat, 
                                              hood_long))

Coordinates for Cedarbrae are 43.773136, -79.23947609999999.


11) Retrieving the top 100 venues of Cedarbra within a radius of 500 meters

In [83]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    hood_lat, 
    hood_long, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RBTEPRZM3KZMHSWZCOZELLIEC1YN2ZFSWCVH4FTOXPZZXRZH&client_secret=RQMYMSAHQKFM1F13BWWW4CTKIKIXJRTCMO5JCJJTCK4FBDYL&v=20180605&ll=43.773136,-79.23947609999999&radius=500&limit=100'

In [84]:
results = requests.get(url).json()

12) Defining the function for retrieving certain portions of JSON.

In [85]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

13) Normalizing the JSON data and converting into a pandas dataframe

In [86]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Federick Restaurant,Hakka Restaurant,43.774697,-79.241142
1,Drupati's Roti & Doubles,Caribbean Restaurant,43.775222,-79.241678
2,Centennial Recreation Centre,Athletics & Sports,43.774593,-79.236500
3,Thai One On,Thai Restaurant,43.774468,-79.241268
4,TD Canada Trust,Bank,43.774952,-79.241343


In [87]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


14) Defining function, and creating dataframe for our examination of Scarborough

In [88]:
radius = 5
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

15) Finding venues within specified distance from our chosen borough

In [89]:
scar_venues = getNearbyVenues(names=scar_data['Neighborhood'],
                                   latitudes=scar_data['Latitude'],
                                   longitudes=scar_data['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [90]:
print(scar_venues.shape)
scar_venues.head()

(86, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


16) Grouping results by neighborhood and counting categories within area

In [91]:
scar_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Clarks Corners, Sullivan, Tam O'Shanter",11,11,11,11,11,11
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"East Birchmount Park, Ionview, Kennedy Park",4,4,4,4,4,4


In [92]:
print('There are {} unique categories.'.format(len(scar_venues['Venue Category'].unique())))

There are 49 unique categories.


16) Creating dummy variables for onehot encoding and displaying resulting dataframe

In [93]:
scar_onehot = pd.get_dummies(scar_venues[['Venue Category']], prefix="", prefix_sep="")
scar_onehot['Neighborhood'] = scar_venues['Neighborhood']
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]
scar_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gaming Cafe,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [94]:
scar_onehot.shape

(86, 50)

17) Cleaning dataframe after onehot encoding

In [95]:
scar_grouped = scar_onehot.groupby('Neighborhood').mean().reset_index()
scar_grouped.head(5)

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gaming Cafe,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.2,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.2,0.0,0.2,0.0,0.20,0.000000,0.000000,0.0,0.0
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0
2,"Birch Cliff, Cliffside West",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.25,0.000000,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.000000,0.0,0.25,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.25,0.000000,0.000000,0.0,0.0
3,Cedarbrae,0.0,0.142857,0.0,0.142857,0.142857,0.0,0.0,0.0,0.000000,0.000000,0.00,0.142857,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.142857,0.0,0.00,0.0,0.142857,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.142857,0.0,0.0
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000000,0.0,0.222222,0.000000,0.0,0.0,0.0,0.222222,0.111111,0.00,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.111111,0.000000,0.0,0.00,0.0,0.000000,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.111111,0.000000,0.0,0.0


In [96]:
scar_grouped.shape

(16, 50)

18) Sorting frequency distribution by neighborhood in descending order

In [97]:
num_top_venues = 5

for hood in scar_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scar_grouped[scar_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0          Lounge   0.2
1    Skating Rink   0.2
2  Breakfast Spot   0.2
3  Sandwich Place   0.2
4      Print Shop   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0                 Park  0.67
1           Playground  0.33
2  American Restaurant  0.00
3            Pet Store  0.00
4    Korean Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4    American Restaurant  0.00


----Cedarbrae----
                venue  freq
0              Bakery  0.14
1                Bank  0.14
2     Thai Restaurant  0.14
3  Athletics & Sports  0.14
4    Hakka Restaurant  0.14


----Clairlea, Golden Mile, Oakridge----
          venue  freq
0        Bakery  0.22
1      Bus Line  0.22
2  Intersection  0.11
3  Soccer Field  0.11
4   Bus Station  0.11


----Clark

19) Creating function to determine the most common venues

In [98]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

20) Creating sorted dataframe for desired query

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scar_grouped['Neighborhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Sandwich Place,Print Shop,Lounge,Breakfast Spot,Vietnamese Restaurant,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Vietnamese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium
2,"Birch Cliff, Cliffside West",Café,General Entertainment,Skating Rink,College Stadium,Caribbean Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Hakka Restaurant,Caribbean Restaurant,Vietnamese Restaurant,College Stadium,General Entertainment
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Park,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint


21) Preparing and fitting data for kmeans  

In [100]:
kclusters = 5

scar_grouped_clustering = scar_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 3, 1], dtype=int32)

22) Inserting Cluster Labels column and assigning names to the resulting clusters

In [101]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

label_names = {0.0:'Bars', 1.0:'Entertainment', 2.0:'Casual Dining', 3.0:'Coffee & Shopping', 4.0:'Groceries'}

neighborhoods_venues_sorted['Cluster Labels'].replace(label_names, inplace=True)

scar_merged = scar_data

scar_merged = scar_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scar_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944,Casual Dining,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,General Entertainment,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,College Stadium
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605,Bars,Bar,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887,Entertainment,Electronics Store,Moving Target,Intersection,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.771,-79.2169,Coffee & Shopping,Coffee Shop,Indian Restaurant,Korean Restaurant,Chinese Restaurant,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395,Entertainment,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Hakka Restaurant,Caribbean Restaurant,Vietnamese Restaurant,College Stadium,General Entertainment


22) Mapping clusters of Foursquare results within the borough of Scarborough and assigning label based on most common venue

In [102]:
map_clusters = folium.Map(location=[la, lo], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], scar_merged['Longitude'], scar_merged['Neighborhood'], scar_merged['Cluster Labels']):
    label = folium.Popup(str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
        
map_clusters

23) Breakdown of the venues within each cluster

In [103]:
scar_merged.loc[scar_merged['Cluster Labels'] == 'Bars', scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,Bars,Bar,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


In [104]:
scar_merged.loc[scar_merged['Cluster Labels'] == 'Entertainment', scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,Entertainment,Electronics Store,Moving Target,Intersection,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Fast Food Restaurant
4,Scarborough,Entertainment,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Hakka Restaurant,Caribbean Restaurant,Vietnamese Restaurant,College Stadium,General Entertainment
7,Scarborough,Entertainment,Bakery,Bus Line,Park,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint
8,Scarborough,Entertainment,American Restaurant,Motel,Chinese Restaurant,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
9,Scarborough,Entertainment,Café,General Entertainment,Skating Rink,College Stadium,Caribbean Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
10,Scarborough,Entertainment,Indian Restaurant,Pet Store,Chinese Restaurant,Gaming Cafe,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
11,Scarborough,Entertainment,Vietnamese Restaurant,Auto Garage,Shopping Mall,Sandwich Place,Breakfast Spot,Middle Eastern Restaurant,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant
12,Scarborough,Entertainment,Skating Rink,Sandwich Place,Print Shop,Lounge,Breakfast Spot,Vietnamese Restaurant,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
13,Scarborough,Entertainment,Pizza Place,Fried Chicken Joint,Thai Restaurant,Fast Food Restaurant,Bank,Rental Car Location,Italian Restaurant,Pharmacy,Noodle House,Chinese Restaurant
14,Scarborough,Entertainment,Park,Playground,Vietnamese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium


In [105]:
scar_merged.loc[scar_merged['Cluster Labels'] == 'Casual Dining', scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,Casual Dining,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,General Entertainment,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,College Stadium


In [106]:
scar_merged.loc[scar_merged['Cluster Labels'] == 'Coffee & Shopping', scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,Coffee & Shopping,Coffee Shop,Indian Restaurant,Korean Restaurant,Chinese Restaurant,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,Scarborough,Coffee & Shopping,Discount Store,Department Store,Bus Station,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant


In [107]:
scar_merged.loc[scar_merged['Cluster Labels'] == 'Groceries', scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,Groceries,Grocery Store,Playground,Caribbean Restaurant,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
